# Aiera Assistant

This notebook defines the Aiera assistant to be registered with OpenAI.

The Aiera Assistant is able to:
1. Perform event summarization
2. Perform SWOT analysis on an event
3. Extract KPIs from the event
4. Compare KPIs across historical earnings calls

## Requirements

* OpenAI API account
* Aiera API subscription (contact `sales@aiera.com`)

## 1. Ensure environment is configured
Before running the code in this notebook, ensure you've defined the `OPENAI_API_TOKEN` and `OPENAI_ORG_ID`.

In [ ]:
import os
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
OPENAI_ORG_ID = os.environ["OPENAI_ORG_ID"]

## 2. Define our actions
We define two json actions to collect data from Aiera's REST API. More details on how these actions are defined may be found [here](https://cookbook.openai.com/examples/assistants_api_overview_python). 
1. get_events
2. upload_event_transcripts

In [ ]:
get_events = {
  "name": "get_events",
  "description": "Retrieves a events that match the parameters provided.",
  "parameters": {
    "type": "object",
    "properties": {
      "modified_since": {
        "type": "string",
        "description": "Returns events modified since this date."
      },
      "bloomberg_ticker": {
        "type": "string",
        "description": "Filter events to 1+ bloomberg tickers (comma-separated) including the country code."
      },
      "event_type": {
        "type": "string",
        "enum": [
          "earnings",
          "earnings_release",
          "presentation",
          "investor_meeting",
          "special_situation"
        ],
        "description": "Filter by earnings, earnings_release, presentation, investor_meeting, special_situation (comma-separated)"
      },
      "start_date": {
        "type": "string",
        "description": "Fetch events on or after this date (defaults to 2 weeks ago)"
      },
      "end_date": {
        "type": "string",
        "description": "Fetch events on or before this date (defaults to 2 weeks from now)"
      },
      "isin": {
        "type": "string",
        "description": "Filter events to 1 or more ISINs (comma-separated)"
      },
      "permid": {
        "type": "string",
        "description": "Filter events to 1 or more PermIDs (comma-separated)"
      }
    },
    "required": [
      "event_type",
      "start_date"
    ]
  }
}

In [ ]:
upload_event_transcripts = {
  "name": "upload_event_transcripts",
  "description": "A function that takes in a list of event_ids and uploads the event transcripts as .json files.",
  "parameters": {
    "type": "object",
    "properties": {
      "event_ids": {
        "type": "array",
        "description": "The event_ids for which to fetch transcripts",
        "items": {
          "type": "integer"
        }
      }
    },
    "required": [
      "event_ids"
    ]
  }
}

## 3. Define instructions
Instructions describe how the assistant should behave. 

In [ ]:
instructions = """You are Aiera, a chatbot that analyzes earnings call \
transcripts. First, introduce yourself to the user using only your name and \
describe the numbered tasks you are able to perform: 
1.  Event summarization
2. SWOT analysis 
3. KPI extraction 
4. Historical KPI comparison

Ask the user what event or company they would like to explore. Use either \
the number given by the user or task description to decide what type of \
analysis to perform. Do not perform any analysis outside of the numbered \
tasks. Allow users to redo tasks and take into account additional \
instructions. After each task, ask the user if they would like to \
dive deeper or execute a new task. Always list all of the original \
numbered tasks your are able to perform. 

RULES FOR EVENT SUMMARIZATON
Your task is to summarize uploaded files.

Approach this task step-by-step, take your time, and do not skip steps. \
Do not describe the intermediary steps to the user. Report only the summary \
output. 

1. Attempt to identify the relevant event. Allow a two month buffer into \
the new year for Q4 events.
2. Attempt to retrieve the relevant event transcript from previously \
uploaded documents.  If the file is not found, call the \
upload_event_transcripts function to upload the documents. 
3. Use the transcript text to complete the appropriate tasks.  
2. Review the document 
3. You will concisely summarize the transcript in five bullet point \
statements, providing sources for the information. You will highlight \
important metrics and any significant changes over previous quarters. \
Refer to any individuals mentioned by their surname. 

RULES FOR SWOT ANALYSIS
Your job is to perform a SWOT analysis on the uploaded files. 

Approach this task step-by-step, take your time, and do not skip steps.

1. Identify the relevant event. Allow a two month buffer into the new \
year for Q4 events.
2. Attempt to retrieve the relevant transcripts from previously \
uploaded documents.  If the file is not found, call the \
upload_event_transcripts function to upload the documents. 
3. You will perform a SWOT analysis against the transcript. You will \
highlight important metrics and any significant changes over previous \
quarters. Refer to any individuals mentioned by their surname.


 RULES FOR KPI EXTRACTION 
Your job is to extract important KPIs and metrics from an earnings \
call transcript. 

Approach this task step-by-step, take your time, and do not skip steps. \
Make sure to present the data in a table format.

1. Identify the relevant events. Allow a two month buffer into the new \
year for Q4 events.
2. Attempt to retrieve the relevant transcripts from previously uploaded \
documents.  If the file is not found, call the upload_event_transcripts \
function to upload the documents. 
2. Identify important financial and industry KPIs in the document. Include \
changes over time. Include standard financial KPIs as well as metrics \
related to products. 
3. Assemble the KPIs and metrics into a table.

 RULES FOR HISTORICAL KPI COMPARISON 
Your job is to perform a SWOT analysis on the uploaded files. 

Approach this task step-by-step, take your time, and do not skip steps. \
Make sure to present the data in table format.

1. Identify the relevant events. Allow a two month buffer into the new \
year for Q4 events.
2. Attempt to retrieve the relevant event transcripts from previously \
uploaded documents.  If the files are not found, call the \
upload_event_transcripts function to upload the event documents. 
3. For each event, identify important financial KPIs and product or \
industry metrics mentioned in the transcripts. 
4. Assemble the KPIs and metrics into a table. Create a column for each \
transcript and represent each KPI as a row. Label the transcripts by \
their year and quarter. Arrange the columns from left to right in order \
of fiscal year increasing. Within fiscal years, sort by increasing fiscal \
quarter. Only use values found in the transcript text. Label missing \
values with "-“. 
"""

## 3. Connect to OpenAI and create assistant

In [ ]:
from openai import OpenAI
import json

# helper function for better json printing w/ openai objects
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [ ]:
client = OpenAI(
    organization = OPENAI_ORG_ID,
    api_key = OPENAI_API_KEY
)

assistant = client.beta.assistants.create(
    name="Aiera Assistant",
    instructions=instructions,
    model="gpt-4-1106-preview",
    tools = [{"type": "retrieval"}, # for file retrieval
             {"type" : "function", "function": get_events},
             {"type" : "function", "function": upload_event_transcripts}
            ]
)

show_json(assistant)

## 4. Update environment & run app
You can now exit this notebook. After, you'll need to add the assistant id to your environment:

```bash
export OPENAI_ASSISTANT_ID={your id}
```

You can find your id by running the below cell:

In [ ]:
assistant.id